In [1]:
import pandas as pd
import os
import numpy as np

## Content API data

In [3]:
DATA_DIR = os.getenv("DATA_DIR")
content_api = os.path.join(DATA_DIR, "content_api")
files = [os.path.join(content_api, date, "content_json.csv.gz") for date in ['07-02-19','22-02-19']]
files

['/Users/felisialoukou/Documents/govuk-network-embedding/data/content_api/07-02-19/content_json.csv.gz',
 '/Users/felisialoukou/Documents/govuk-network-embedding/data/content_api/22-02-19/content_json.csv.gz']

In [5]:
dfj = pd.concat([pd.read_csv(f,compression='gzip') for f in files], ignore_index=True)

In [18]:
dfj.dropna(subset = ["content_id"], inplace=True)

In [39]:
dfj.shape

(59919, 13)

In [38]:
dfj.drop_duplicates("content_id", inplace=True)

In [40]:
dfj.to_csv(os.path.join(content_api, "merged_content_json.csv.gz"), compression="gzip", index=False)

## edge data

In [9]:
processed_network = os.path.join(DATA_DIR, "processed_network")
edges_file = os.path.join(processed_network, "edges_graphsagetest_feb_01_18_doo_min15weight.csv.gz")
edges = pd.read_csv(edges_file, compression="gzip")
edges.head()

,src_node,dest_node,weight
0,/government/publications/guidance-for-dependan...,/visa-fees,66
1,/visa-fees,/find-a-visa-application-centre,1164
2,/entering-staying-uk/family-visas,/uk-family-visa,377
3,/uk-family-visa,/uk-family-visa/partner-spouse,26227
4,/uk-family-visa/partner-spouse,/government/publications/application-for-uk-vi...,2733


In [33]:
pages = set(list(edges.src_node) + list(edges.dest_node))
len(pages), len(pages.difference(set(dfj.url.values)))

(66619, 0)

In [21]:
url_id = dict(zip(dfj.url, dfj.content_id))

In [22]:
edges['src_cid'] = edges['src_node'].map(url_id)
edges['dest_cid'] = edges['dest_node'].map(url_id)

In [26]:
edges[edges['src_cid'].isna()].shape[0] + edges[edges['dest_cid'].isna()].shape[0]

4392

In [28]:
edges[(~edges['src_cid'].isna()) & (~edges['dest_cid'].isna())].shape

(213446, 5)

### New edges, have text

In [32]:
edges = edges[(~edges['src_cid'].isna()) & (~edges['dest_cid'].isna())].copy(deep=True)

In [42]:
edges_file_filtered = os.path.join(processed_network, "edges_graphsagetest_feb_01_18_doo_min15weight_wtext.csv.gz")
edges.to_csv(edges_file_filtered, compression="gzip", index=False)

## Compute numeric node ids to be used with stellargraph implementation

In [34]:
node_id = {}
id_count = 0
for node in set(list(edges.src_node) + list(edges.dest_node)):
    if node not in node_id.keys():
        node_id[node] = id_count
        id_count+=1

In [ ]:
edges['source'] = edges['src_node'].map(node_id)
edges['target'] = edges['dest_node'].map(node_id)